In [6]:

import numpy as np 
import pandas as pd 
import warnings 
warnings.filterwarnings("ignore")
import tensorflow as tf 


print(tf.__version__)

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM,Bidirectional
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import RepeatVector
from tensorflow.keras import Input
from tensorflow.keras.layers import Flatten, Reshape
from tensorflow.keras import Model
from tensorflow.keras.layers import Conv2D,concatenate
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras import initializers
import tensorflow.keras.backend as K


from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Activation
from keras.layers.advanced_activations import LeakyReLU
from tensorflow.keras.optimizers import Adam

import math
import pickle5 as pickle
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


2.2.0


In [7]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="2"
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession


gpus = tf.config.experimental.list_logical_devices('GPU')
print(gpus)
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu,True)
    except:
        print('checking gpu error')
print('checking GPUs')

config=tf.compat.v1.ConfigProto()
# config.gpu_options.visible_device_list= '0,1'
config.gpu_options.allow_growth=True

config.log_device_placement=True
# config.visible_device_list =2
sess=tf.compat.v1.Session(config=config)

tf.compat.v1.keras.backend.set_session(sess)




[LogicalDevice(name='/device:GPU:0', device_type='GPU')]
checking gpu error
checking GPUs
Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: TITAN X (Pascal), pci bus id: 0000:09:00.0, compute capability: 6.1



In [11]:
## data setting
sample_size = 1000
num_per_channel=10
x_column=25
y_column=25
channels=3
sample_shape= (num_per_channel, x_column, y_column, channels)
z_dim = 100

In [13]:
def generator(img_shape, z_dim):
    model = Sequential()
    model.add(Dense(128, input_dim=z_dim))
    model.add(LeakyReLU(alpha=0.01))
    model.add(Dense(num_per_channel*x_column*y_column*channels, activation='tanh'))
    model.add(Reshape(img_shape))
    z = Input(shape=(z_dim,))
    img = model(z)
    return Model(z, img)

def discriminator(img_shape):
    model = Sequential()
    model.add(Flatten(input_shape=img_shape))
    model.add(Dense(128))
    model.add(LeakyReLU(alpha=0.01))
    model.add(Dense(1, activation='sigmoid'))
    img = Input(shape=img_shape)
    prediction = model(img)
    return Model(img, prediction)

discriminator = discriminator(sample_shape)
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
generator = generator(sample_shape, z_dim)
z = Input(shape=(100,))
img = generator(z)
discriminator.trainable = False
prediction = discriminator(img)
combined = Model(z, prediction)
combined.compile(loss='binary_crossentropy', optimizer=Adam())

In [15]:
# training

(10, 25, 25, 3)


In [18]:

losses = []
accuracies = []
def train(iterations, batch_size, sample_interval):
    gen_images = []
    
    # Assign X_train to X_train_0 for augment non-cactus images
    # Assign X_train to X_train_1 for augment cactus images

    X_train = np.random.rand(sample_size, num_per_channel, x_column, y_column, channels)
    real = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for iteration in range(iterations):
       
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        imgs = X_train[idx]

        z = np.random.normal(0, 1, (batch_size, 100))
        gen_imgs = generator.predict(z)

        d_loss_real = discriminator.train_on_batch(imgs, real)
        d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        z = np.random.normal(0, 1, (batch_size, 100))
        gen_imgs = generator.predict(z)
        g_loss = combined.train_on_batch(z, real)

        if iteration % sample_interval == 0:
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (iteration, d_loss[0], 100*d_loss[1], g_loss))
            losses.append((d_loss[0], g_loss))
            accuracies.append(100*d_loss[1])
            gen_images.append(sample_images(iteration))
    return gen_images

In [19]:


def sample_images(iteration, image_grid_rows=4, image_grid_columns=4):

    z = np.random.normal(0, 1, 
              (image_grid_rows * image_grid_columns, z_dim))

    gen_imgs = generator.predict(z)
    gen_imgs = 0.5 * gen_imgs + 0.5
            
    return gen_imgs



In [22]:
# Set iterations at least 10000 for good results
iterations = 5000
batch_size = 128
sample_interval = 100
gen_imgs = train(iterations, batch_size, sample_interval)

0 [D loss: 0.141338, acc.: 100.00%] [G loss: 1.471242]
100 [D loss: 0.143022, acc.: 100.00%] [G loss: 1.520105]
200 [D loss: 0.111269, acc.: 100.00%] [G loss: 1.831884]
300 [D loss: 0.015712, acc.: 100.00%] [G loss: 3.568152]
400 [D loss: 0.016181, acc.: 100.00%] [G loss: 3.658872]
500 [D loss: 0.446788, acc.: 50.00%] [G loss: 1.102878]
600 [D loss: 0.090484, acc.: 100.00%] [G loss: 1.936445]
700 [D loss: 0.075122, acc.: 100.00%] [G loss: 2.139253]
800 [D loss: 0.074992, acc.: 100.00%] [G loss: 2.140151]
900 [D loss: 0.086324, acc.: 100.00%] [G loss: 2.146873]
1000 [D loss: 0.071163, acc.: 100.00%] [G loss: 2.286811]
1100 [D loss: 0.016131, acc.: 100.00%] [G loss: 4.007945]
1200 [D loss: 0.072102, acc.: 100.00%] [G loss: 2.386014]
1300 [D loss: 0.081814, acc.: 100.00%] [G loss: 2.430460]
1400 [D loss: 0.023009, acc.: 100.00%] [G loss: 3.133158]
1500 [D loss: 0.235325, acc.: 98.44%] [G loss: 1.516053]
1600 [D loss: 0.052348, acc.: 100.00%] [G loss: 2.862943]
1700 [D loss: 0.022226, acc.